In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os 

from datetime import datetime

import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)


In [ ]:
dir_path = os.getcwd()
dir_path

In [ ]:
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)
savedir = "models/Tensor_CIFAR_Sparse_model/" + "run-{}/".format(now)

In [ ]:
print(savedir)

In [ ]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [ ]:
cifar10_1 = unpickle('datasets/cifar-10/data_batch_1')

In [ ]:
cifar10_dataset_folder_path = 'datasets/cifar-10/'

In [ ]:
list(cifar10_1.keys())

In [ ]:
len(cifar10_1[b'data']),len(cifar10_1[b'labels'])

In [ ]:
cifar10_1[b'batch_label']

In [ ]:
cifar10_Label_names = unpickle('datasets/cifar-10/batches.meta')
cifar10_Label_names[b'label_names']

In [ ]:
label_names = []
for x in cifar10_Label_names[b'label_names']:
    label = x.decode()
    label_names.append(label)

In [ ]:
label_names

In [ ]:
def load_cfar10_batch(cifar10_dataset_folder_path, batch_id):
    with open(cifar10_dataset_folder_path + '/data_batch_' + str(batch_id), mode='rb') as file:
        # note the encoding type is 'latin1'
        batch = pickle.load(file, encoding='latin1')
        
    features = batch['data'].reshape((len(batch['data']), 3, 32, 32)).transpose(0, 2, 3, 1)
    labels = batch['labels']
        
    return features, labels

In [ ]:
def display_stats(cifar10_dataset_folder_path, batch_id, sample_id):
    features, labels = load_cfar10_batch(cifar10_dataset_folder_path, batch_id)
    
    if not (0 <= sample_id < len(features)):
        print('{} samples in batch {}.  {} is out of range.'.format(len(features), batch_id, sample_id))
        return None

    print('\nStats of batch #{}:'.format(batch_id))
    print('# of Samples: {}\n'.format(len(features)))
    
    label_counts = dict(zip(*np.unique(labels, return_counts=True)))
    for key, value in label_counts.items():
        print('Label Counts of [{}]({}) : {}'.format(key, label_names[key].upper(), value))
    
    sample_image = features[sample_id]
    sample_label = labels[sample_id]
    
    print('\nExample of Image {}:'.format(sample_id))
    print('Image - Min Value: {} Max Value: {}'.format(sample_image.min(), sample_image.max()))
    print('Image - Shape: {}'.format(sample_image.shape))
    print('Label - Label Id: {} Name: {}'.format(sample_label, label_names[sample_label]))
    
    plt.imshow(sample_image)

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np

# Explore the dataset
batch_id = 3
sample_id = 7000
display_stats(cifar10_dataset_folder_path, batch_id, sample_id)

In [ ]:
def normalize(x):

    min_val = np.min(x)
    max_val = np.max(x)
    x = (x-min_val) / (max_val-min_val)
    
    x = x.astype('float32')
    
    return x

In [ ]:
def one_hot_encode(x):

    encoded = np.zeros((len(x), 10))
    
    for idx, val in enumerate(x):
        encoded[idx][val] = 1
    
    return encoded

In [ ]:
def _preprocess_and_save(normalize, one_hot_encode, features, labels, filename):
    features = normalize(features)
    labels = one_hot_encode(labels)
    
    pickle.dump((features, labels), open(filename, 'wb'))

In [ ]:
def preprocess_and_save_data(cifar10_dataset_folder_path, normalize, one_hot_encode):
    n_batches = 5
    valid_features = []
    valid_labels = []
    all_features = []
    all_labels = []

    for batch_i in range(1, n_batches + 1):
        features, labels = load_cfar10_batch(cifar10_dataset_folder_path, batch_i)
        
        # find index to be the point as validation data in the whole dataset of the batch (10%)
        index_of_validation = int(len(features) * 0.1)

        valid_features.extend(features[-index_of_validation:])
        valid_labels.extend(labels[-index_of_validation:])
        all_features.extend(features[:-index_of_validation])
        all_labels.extend(labels[:-index_of_validation])

    # preprocess the all stacked validation dataset
    _preprocess_and_save(normalize, one_hot_encode,
                         np.array(valid_features), np.array(valid_labels),
                         'datasets/preprocess_validation.p')
                                 
    _preprocess_and_save(normalize, one_hot_encode,
                         np.array(all_features), np.array(all_labels),
                         'datasets/preprocess_all.p')
    

In [ ]:
preprocess_and_save_data(cifar10_dataset_folder_path, normalize, one_hot_encode)

In [ ]:
valid_features, valid_labels = pickle.load(open('datasets/preprocess_validation.p', mode='rb'))
print(valid_features.dtype)
print(valid_features.shape)

In [ ]:
train_features, train_labels = pickle.load(open('datasets/preprocess_all.p', mode='rb'))
print(train_features.dtype)
print(train_features.shape)

In [ ]:
def cnn_model_fn(features, labels, mode):
    
    with tf.name_scope('model_input') as scope:
        input_layer = tf.reshape(features, [-1, 32, 32, 3], name="input")
        
    with tf.name_scope('model_conv1') as scope:    
        conv1 = tf.layers.conv2d(inputs=input_layer, filters=64, kernel_size=[5, 5], 
                                 padding="same", activation=tf.nn.relu6, 
                                 trainable=mode == tf.estimator.ModeKeys.TRAIN)
        pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=(2,2))
        
    with tf.name_scope('model_conv2') as scope:
        conv2 = tf.layers.conv2d(inputs=pool1, filters=128, kernel_size=[5, 5], 
                                 padding="same", activation=tf.nn.relu6, 
                                 trainable=mode == tf.estimator.ModeKeys.TRAIN)
        pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=(2,2))
    
    with tf.name_scope('model_dense') as scope:
        pool2_flat = tf.reshape(pool2, [-1, 8 * 8 * 128]) 
    
        dense = tf.layers.dense(inputs=pool2_flat, units=1024, 
                                activation=tf.nn.relu6, 
                                trainable=mode == tf.estimator.ModeKeys.TRAIN)
        
        dropout = tf.layers.dropout(inputs=dense, rate=0.25, training=mode == tf.estimator.ModeKeys.TRAIN)
        
    with tf.name_scope('model_output') as scope:    
        logits = tf.layers.dense(inputs=dropout, units=10, trainable=mode == tf.estimator.ModeKeys.TRAIN)
        # shape should be:[-1, 10]
    
    predictions = {
      "classes": tf.argmax(input=logits, axis=1),
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    # PREDICT mode
    if mode == tf.estimator.ModeKeys.PREDICT:
        
        export_outputs = {
            'predict_output': tf.estimator.export.PredictOutput(predictions)
        }
        
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions, export_outputs=export_outputs)
    
    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.softmax_cross_entropy(onehot_labels=labels, logits=logits)
    
    
    # TRAIN mode
    if mode == tf.estimator.ModeKeys.TRAIN:
        g = tf.get_default_graph()
        tf.contrib.quantize.create_training_graph(input_graph=g, quant_delay=6000) 
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # EVAL mode
    g = tf.get_default_graph()
    tf.contrib.quantize.create_eval_graph(input_graph=g)
    labels = tf.argmax(labels, axis=1)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])
    }
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

    

In [ ]:
#see for options here: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/protobuf/config.proto
#sess_config = tf.ConfigProto(gpu_options=gpu_options)

run_cfg = tf.estimator.RunConfig( 
    model_dir=savedir,
    tf_random_seed=2,
    save_summary_steps=2,
    #session_config = sess_config,
    save_checkpoints_steps=100,
    keep_checkpoint_max=1) 

In [ ]:
# Instantiate the Estimator
cifar10_classifier = tf.estimator.Estimator(
        model_fn=cnn_model_fn, config=run_cfg) 

In [ ]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}

logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=100)

In [ ]:
def fit_all_batches(xEpochs):
    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x=train_features,
        y=train_labels,
        batch_size=32,
        num_epochs=xEpochs,
        shuffle=False)
        
    # train one step and display the probabilties
    cifar10_classifier.train(
        input_fn=train_input_fn,
        steps=None, 
        hooks=[logging_hook]) 

In [ ]:
fit_all_batches(100)

In [ ]:
defaultGraph = tf.get_default_graph()
    
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=valid_features,
    y=valid_labels,
    num_epochs=1,
    shuffle=False)

eval_results = cifar10_classifier.evaluate(input_fn=eval_input_fn,
                                          hooks=[logging_hook])
            
print(eval_results)


In [ ]:
sample_image = valid_features[3:4]

In [ ]:
sample_image.shape

In [ ]:
label_names

In [ ]:
def predictions(nfeature):
    
    pred_input_fn = tf.estimator.inputs.numpy_input_fn(
        x=valid_features[nfeature:nfeature+1],
        y=valid_labels[nfeature:nfeature+1],
        num_epochs=1,
        shuffle=False)
    
    pred_results = cifar10_classifier.predict(
            input_fn=pred_input_fn,
            hooks=[logging_hook])
    
    results = list(pred_results)
    results = results[0]
 
    y_pred_flat = results['probabilities']
    
    y_pred = np.column_stack((label_names, y_pred_flat))
    print(y_pred)
    plt.imshow(valid_features[nfeature])

In [ ]:
predictions(6)

In [ ]:
[print(n.name) for n in defaultGraph.as_graph_def().node]

In [ ]:
def serving_input_receiver_fn():

    feature_tensor = tf.placeholder(tf.float32, [None, 32,32,3])
    return tf.estimator.export.TensorServingInputReceiver(feature_tensor, {'input': feature_tensor})

In [ ]:
#cifar10_classifier.export_savedmodel(savedir, serving_input_receiver_fn,
#    strip_default_attrs=True)


In [ ]:
input_receiver_fn_map={
    tf.estimator.ModeKeys.PREDICT: serving_input_receiver_fn,
    }

cifar10_classifier.experimental_export_all_saved_models(
    savedir, 
    input_receiver_fn_map)